# Regressão Linear Múltipla
---

<img src="https://selecao.letscode.com.br/favicon.png" width="40px" style="position: absolute; top: 15px; right: 20px; border-radius: 5px;" />

<img src="https://selecao.letscode.com.br/favicon.png" width="40px" style="position: absolute; top: 15px; right: 20px; border-radius: 5px;" />

1. Análise Exploratória dos Dados
2. Criação do Modelo de Regressão Linear Múltipla  
    2.1. Sklearn  
    2.2. StatsModels  
3. Remoção de Outliers
4. Considerando dados categóricos
3. Retirando Variáveis
4. Transformação de Variáveis
5. Na prática: Divisão Treino/Teste
6. Exercício

In [64]:
# Importação das bibliotecas


In [223]:
# Carregando dataset


## Análise Exploratória dos Dados

> Como os dados se relacionam?

In [1]:
# Matriz de correlação


## Definir a Regressão Linear Múltipla

In [77]:
# X: Length1 Length2 Length3 Height	Width
# y: Weight

---
### Scikit Learn

In [ ]:
# Montando a tabela de coeficientes


#### Regressão Linear Simples

$$
y = \beta_0 + \beta_1 \cdot x
$$

#### Regressão Linear Múltipla

$$
    y_i = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5
$$

#### Predição para todos os dados do Dataset

### Statsmodels

### Análise do Erro da Regressão

#### 1. A média do erro deve ser zero

#### 2. A distribuição dos erros deve seguir uma distribuição normal

#### 3. Erro deve ser descorrelacionado

## Removendo Outliers

## Modelo de Regressão sem Outliers
---

### Sklearn

### Statsmodels

## Considerando os Dados Categóricos
---

## Avaliando o modelo com a presença dos Dados Categóricos

### Sklearn

### LinearRegression

### Statsmodels

### Remoção de Variáveis Desnecessárias
---

### Sklearn

### Statsmodels

## Transformação de Variáveis
---

### Transformação log & Transformação log-log (duplo)

### Separando as variáveis explicativas e a variável resposta

### Sklearn

### Statsmodels

### Removendo as espécies

### Sklearn

### Statsmodels

### Sklearn

## Treino/Teste
---

1. Processo de treinamento do nosso modelo
2. Processo de teste do nosso modelo

#### Divisão treino-teste

- 70% dos dados para treino
- 30% dos dados para teste

---